<a href="https://colab.research.google.com/github/mcatrinque/bert_potagging/blob/main/bert_postagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avaliação de POS Tagging Baseado em Bert


## Introdução

Neste notebook, abordaremos a tarefa de Part-of-Speech (POS) Tagging para a língua Portuguesa.
O objetivo principal é explorar e implementar um modelo de POS Tagging utilizando o corpus Mac-Morpho, que foi desenvolvido pelo grupo NILC da ICMC USP. Este corpus fornece dados essenciais para treinamento, validação e teste de modelos preditivos capazes de classificar a classe gramatical de palavras em Português.
Para mais informações sobre as classes gramaticais disponíveis, consulte o manual do Mac-Morpho [aqui](http://nilc.icmc.usp.br/macmorpho/macmorpho-manual.pdf).

In [1]:
!pip install wget
!pip install transformers
!pip install torch
!pip install seqeval
!pip install plotly

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=e5b49165446528b69c7fdd189c6c960839b1b725067f6ee188e7517416c11277
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ffa82c17e4a0e533bbba7a84d2375bb1314f3ad5d40c33f19de6b4a39eca1757
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
import wget

import torch

import pandas as pd
import numpy as np

from tqdm import tqdm

from transformers import BertTokenizer, BertForTokenClassification, pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, precision_score, recall_score, jaccard_score, accuracy_score, classification_report, confusion_matrix, jaccard_score
from seqeval.metrics import accuracy_score as seq_accuracy_score

from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tag import hmm

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

### Corpus Mac-Morpho

O corpus Mac-Morpho pode ser acessado [aqui](http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz).
Ele oferece uma rica fonte de dados para aprimorar a compreensão e a precisão dos modelos de POS Tagging em Português.

In [3]:
url = "http://nilc.icmc.usp.br/macmorpho/macmorpho-v3.tgz"
wget.download(url, "macmorpho-v3.tgz")
!tar -xzvf macmorpho-v3.tgz
!rm macmorpho-v3.tgz

macmorpho-dev.txt
macmorpho-test.txt
macmorpho-train.txt


## Metodologia

### Carregamento do Corpus Mac-Morpho
Utilizando a biblioteca `wget`, foi feito o download do corpus Mac-Morpho a partir do link fornecido. Em seguida, o arquivo foi descompactado, e a função `read_corpus` foi criada para ler o corpus a partir do arquivo e estruturar os dados em um DataFrame do Pandas.

In [4]:
# Função para ler o corpus a partir de um arquivo
def read_corpus(arquivo):
    with open(arquivo, "r", encoding="utf-8") as file:
        texto = file.read()
    sentencas = texto.split('\n')

    tokens = [token.strip() for token in texto.split()]
    tuplas_palavras = [tuple(token.split('_') + [i]) for i, sentenca in enumerate(sentencas) for token in sentenca.split()]

    df = pd.DataFrame(tuplas_palavras, columns=['word', 'pos_tag', 'sentence'])
    return df

# Carregando os conjuntos de dados de treinamento, validação e teste
macmorpho_train = read_corpus("macmorpho-train.txt")
macmorpho_dev = read_corpus("macmorpho-dev.txt")
macmorpho_test = read_corpus("macmorpho-test.txt")
macmorpho_test

,word,pos_tag,sentence
0,Salto,N,0
1,sete,ADJ,0
2,O,ART,1
3,grande,ADJ,1
4,assunto,N,1
...,...,...,...
178368,a,ART,9986
178369,antecipação,N,9986
178370,de,PREP,9986
178371,eleições,N,9986


### Carregamento do Tokenizer e Modelo BERT Pré-treinado

Nesta seção, o tokenizer e o modelo BERT específicos para o português são carregados.
O tokenizer é responsável por dividir o texto em tokens, enquanto o modelo BERT é pré-treinado para classificação de tokens, sendo adequado para a tarefa de POS Tagging.
Utilizar modelos pré-treinados permite aproveitar o conhecimento adquirido durante o treinamento em grandes conjuntos de dados, beneficiando-se de padrões linguísticos complexos.

In [5]:
# Carregando o tokenizer e o modelo pré-treinado BERT em português
bert_base = "lisaterumi/postagger-portuguese"
tokenizer_bert = BertTokenizer.from_pretrained(bert_base)
model_bert = BertForTokenClassification.from_pretrained(bert_base)

tokenizer_config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [6]:
# Exibir algumas frases de teste e suas tags reais
for (sentence_group, tag_group), _ in zip(macmorpho_test.groupby('sentence')[['word', 'pos_tag']].apply(lambda group: (group['word'].tolist(), group['pos_tag'].tolist())).head(2), range(2)):
    print(f"Sentence: {sentence_group}, Actual Tags: {tag_group}")

Sentence: ['Salto', 'sete'], Actual Tags: ['N', 'ADJ']
Sentence: ['O', 'grande', 'assunto', 'da', 'semana', 'em', 'Nova', 'York', 'é', 'a', 'edição', 'da', 'revista', '"', 'New', 'Yorker', '"', 'que', 'está', 'nas', 'bancas', '.'], Actual Tags: ['ART', 'ADJ', 'N', 'PREP+ART', 'N', 'PREP', 'NPROP', 'NPROP', 'V', 'ART', 'N', 'PREP+ART', 'N', 'PU', 'NPROP', 'NPROP', 'PU', 'PRO-KS', 'V', 'PREP+ART', 'N', 'PU']


### Previsão de POS Tags com o Modelo BERT
Foi implementada a função `predict_pos_tags_bert` para prever as POS tags utilizando o modelo BERT. O teste da função com uma sentença de exemplo foi realizado para verificar seu funcionamento.

In [7]:
# Função para prever as POS tags com o modelo BERT
def predict_pos_tags_bert(sentence, model, tokenizer):
    tokens = sentence
    with torch.no_grad():
        outputs = model(torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]))
    # Obter as previsões para cada token na sentença
    predictions = torch.argmax(outputs.logits, dim=2).squeeze().tolist()
    if(type(predictions) is int):
        predictions = [predictions]
    # Mapear os rótulos BERT para as palavras na sentença
    predicted_tags = [model.config.id2label[i] for i in predictions]
    return predicted_tags

# Testando a função
sentence_example = macmorpho_test[macmorpho_test['sentence'] == 1]['word'].tolist()
predicted_labels_example = predict_pos_tags_bert(sentence_example, model_bert, tokenizer_bert)

print("Frase:", sentence_example)
print("Rótulos Preditos:", predicted_labels_example)
print("Rótulos Originais:", macmorpho_test[macmorpho_test['sentence'] == 1]['pos_tag'].tolist())

Frase: ['O', 'grande', 'assunto', 'da', 'semana', 'em', 'Nova', 'York', 'é', 'a', 'edição', 'da', 'revista', '"', 'New', 'Yorker', '"', 'que', 'está', 'nas', 'bancas', '.']
Rótulos Preditos: ['<pad>', 'ADJ', 'N', 'PREP+ART', 'N', 'PREP', 'NPROP', 'NPROP', 'V', 'ART', 'N', 'PREP+ART', 'N', 'PU', 'NPROP', 'NPROP', 'PU', 'PRO-KS', 'V', 'PREP+ART', 'ADJ', 'PU']
Rótulos Originais: ['ART', 'ADJ', 'N', 'PREP+ART', 'N', 'PREP', 'NPROP', 'NPROP', 'V', 'ART', 'N', 'PREP+ART', 'N', 'PU', 'NPROP', 'NPROP', 'PU', 'PRO-KS', 'V', 'PREP+ART', 'N', 'PU']


## Processamento das Sentenças e Armazenamento de Previsões

Neste trecho do código, é realizada a iteração sobre as sentenças do conjunto de teste do corpus Mac-Morpho.
Para cada sentença, as POS tags são previstas utilizando a função `predict_pos_tags_bert` implementada anteriormente. As previsões resultantes para cada sentença são então armazenadas na lista `predictions_bert`.
Este processo é crucial para posterior avaliação do modelo e análise das métricas de desempenho.

In [8]:
# Criar uma lista para armazenar todas as previsões
predictions_bert = []

# Iterar sobre as sentenças do conjunto de teste
for sentence_id in tqdm(macmorpho_test['sentence'].unique(), desc="Processing sentences with BERT"):
    sentence_data = macmorpho_test[macmorpho_test['sentence'] == sentence_id]['word']
    predicted_tags = predict_pos_tags_bert(sentence_data, model_bert, tokenizer_bert)
    predictions_bert.append(predicted_tags)

Processing sentences with BERT: 100%|██████████| 9987/9987 [21:12<00:00,  7.85it/s]


## Resultados

O modelo BERT foi utilizado para prever as POS tags em todo o conjunto de teste do Mac-Morpho. As métricas de avaliação, como Acurácia, F1 Score, Precisão, Recall e Jaccard Score, foram calculadas e apresentadas em uma tabela interativa utilizando a biblioteca Plotly.

### Métricas de Avaliação Gerais
As métricas de avaliação geral do modelo foram apresentadas em uma tabela interativa, destacando a Acurácia, F1 Score, Precisão, Recall e Jaccard Score.

In [9]:
flat_predictions = [tag for tags in predictions_bert for tag in tags]
flat_labels = macmorpho_test['pos_tag'].tolist()
class_labels = sorted(set(macmorpho_train['pos_tag'].tolist()))

In [10]:
accuracy = accuracy_score(flat_labels, flat_predictions)
f1 = f1_score(flat_labels, flat_predictions, average='weighted', zero_division=1)
precision = precision_score(flat_labels, flat_predictions, average='weighted', zero_division=1)
recall = recall_score(flat_labels, flat_predictions, average='weighted', zero_division=1)
jaccard = jaccard_score(flat_labels, flat_predictions, average='weighted', zero_division=1)

In [11]:
def plot_evaluation_metrics_table(metric_names, metric_values, title='Métricas de Avaliação', width=500, height=300):
    # Formatar os valores das métricas para exibir apenas duas casas decimais
    formatted_metric_values = [f"{value:.2f}" for value in metric_values]

    # Criar a tabela interativa usando Plotly Graph Objects
    fig = go.Figure(data=[go.Table(
        header=dict(values=["Métrica", "Valor"]),
        cells=dict(values=[metric_names, formatted_metric_values]),
    )])

    # Personalizar a aparência da tabela
    fig.update_layout(
        title_text=title,
        width=width,
        height=height,
        margin=dict(t=50, l=0, r=0, b=0),
        title_x=0.5,
    )

    fig.show()

In [12]:
metric_names = ['Acurácia', 'F1 Score', 'Precisão', 'Recall', 'Jaccard Score']
metric_values = [accuracy, f1, precision, recall, jaccard]
plot_evaluation_metrics_table(metric_names, metric_values)

### Análise por Classe Gramatical
Utilizando o relatório de classificação, foram criados gráficos interativos que apresentam as métricas de Precisão, Recall e F1 Score para cada classe gramatical.

In [13]:
def plot_classification_metrics(classification_report_str):
    # Analisar o relatório de classificação para extrair informações
    lines = classification_report_str.split('\n')
    classes = []
    precision = []
    recall = []
    f1_score = []

    for line in lines[2:-5]:
        row_data = line.split()
        classes.append(row_data[0])
        precision.append(float(row_data[1]))
        recall.append(float(row_data[2]))
        f1_score.append(float(row_data[3]))

    # Criar um gráfico interativo usando Plotly
    fig = make_subplots(rows=1, cols=3, subplot_titles=('Precision', 'Recall', 'F1 Score'))

    fig.add_trace(go.Bar(x=precision, y=classes, orientation='h', name='Precision'), row=1, col=1)
    fig.add_trace(go.Bar(x=recall, y=classes, orientation='h', name='Recall'), row=1, col=2)
    fig.add_trace(go.Bar(x=f1_score, y=classes, orientation='h', name='F1 Score'), row=1, col=3)

    fig.update_layout(title_text='Métricas por Classe Gramatical', showlegend=False)
    fig.show()

In [14]:
classification_rep = classification_report(flat_labels, flat_predictions, zero_division=1)
plot_classification_metrics(classification_rep)

### Coeficiente de Jaccard por Classe Gramatical
O coeficiente de Jaccard foi calculado para cada classe gramatical, e um gráfico interativo foi gerado para visualizar a distribuição desses coeficientes.

In [15]:
def plot_jaccard_scores(class_jaccard):
    # Ordenar as classes gramaticais por coeficiente de Jaccard (da maior para a menor)
    sorted_class_jaccard = sorted(class_jaccard.items(), key=lambda x: x[1], reverse=True)

    # Extrair classes e coeficientes de Jaccard ordenados
    classes = [label for label, _ in sorted_class_jaccard]
    jaccard_scores = [jac for _, jac in sorted_class_jaccard]

    # Criar um gráfico interativo usando Plotly
    fig = go.Figure()
    fig.add_trace(go.Bar(x=classes, y=jaccard_scores, marker_color='skyblue'))

    fig.update_layout(title_text='Coeficiente de Jaccard por Classe Gramatical',
                      xaxis_title='Classe Gramatical',
                      yaxis_title='Coeficiente de Jaccard')
    fig.show()

In [16]:
class_jaccard = {}
for label in class_labels:
    indices = [i for i, tag in enumerate(flat_labels) if tag == label]
    class_jaccard[label] = jaccard_score([flat_labels[i] for i in indices], [flat_predictions[i] for i in indices], average='weighted')

# Ordenar as classes gramaticais por coeficiente de Jaccard (da maior para a menor)
sorted_class_jaccard = sorted(class_jaccard.items(), key=lambda x: x[1], reverse=True)
# Imprimir as classes gramaticais e seus coeficientes de Jaccard
plot_jaccard_scores(class_jaccard)

## Conclusões e Próximos Passos

O processo de POS Tagging em português, utilizando o modelo BERT pré-treinado, revelou resultados robustos e insights valiosos. A análise das métricas e do coeficiente de Jaccard por classe gramatical forneceu uma compreensão aprofundada do desempenho do modelo.

### Destaques dos Resultados:

- **Acurácia e Métricas Gerais:**
  - A Acurácia do modelo alcançou 83.37%, indicando uma performance sólida na classificação de classes gramaticais.
  - O F1 Score, Precisão e Recall atingiram, respectivamente, 85.61%, 90.28%, e 83.37%, evidenciando um equilíbrio entre precisão e recuperação.

- **Coeficiente de Jaccard por Classe Gramatical:**
  - Classes como pontuação (PU), preposição + artigo (PREP+ART) e preposição + pronome (PREP+PRO-KS) alcançaram coeficientes superiores a 90%, destacando sua predição precisa.
  - Algumas classes menos frequentes, como "CUR" (Currency), apresentaram coeficiente zero, sugerindo desafios específicos.

- **Relatório de Classificação:**
  - Classes como pontuação (PU), preposição + artigo (PREP+ART), preposição + pronome (PREP+PRO-KS), e substantivo (N) mostraram alto desempenho com Precision, Recall e F1 Score elevados.

### Considerações Finais:

- O modelo BERT demonstrou eficácia na tarefa de POS Tagging em português, capturando nuances gramaticais complexas.
- A análise por classe gramatical oferece direcionamentos para melhorias, sugerindo ajustes específicos em classes com métricas menos favoráveis.
- A escolha de ajustes finos, expansão do conjunto de dados ou a exploração de arquiteturas mais avançadas pode aprimorar ainda mais o desempenho do modelo.

Esses resultados destacam o potencial e a adaptabilidade dos modelos pré-treinados para tarefas linguísticas específicas em português, incentivando futuras explorações e otimizações.